In [ ]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys, os
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")
    
    # Overlay
    ol = Overlay("FFT.bit")
    ipFFT = ol.FFT_0

    FFT_in = open("FFT_in.txt", "r+")

    # Memory allocate
    inBuffer0 = allocate(shape=(1024,), dtype=np.float64)
    
    FFT_in.seek(0)
    for i in range(1024):
        line = FFT_in.readline()
        inBuffer0[i] = float(line)
    FFT_in.close()
    
    timeKernelStart = time()
    # ============= Set Kernel Argument ==============
    # control
    # 0x00 : Control signals
    #        bit 0  - ap_start (Read/Write/COH)
    #        bit 1  - ap_done (Read/COR)
    #        bit 2  - ap_idle (Read)
    # 0x10 : Data signal of f
    #        bit 31~0 - f[31:0] (Read/Write)
    # 0x14 : Data signal of f
    #        bit 31~0 - f[63:32] (Read/Write)
    # 0x1c : Data signal of logn
    #        bit 31~0 - logn[31:0] (Read/Write)
    logn = 10
    ipFFT.write(0x1C, logn) # Coefficient

    # ================ Data Transfer =================
    ipFFT.write(0x10, inBuffer0.device_address) # 0x10 bit 31:0
    ipFFT.write(0x14, inBuffer0.device_address) # 0x14 bit 63:32

    # =================== Start ======================
    ipFFT.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
    while (ipFFT.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    
    # ================== Compare =====================
    error_cnt = 0
    FFT_out = open("FFT_out.txt", "r+")
    FFT_out.seek(0)
    for i in range(1024):
        line = FFT_out.readline()
        golden = float(line)
        if inBuffer0[i] != golden:
            print(f"Failed, expect value: {golden:.17f}")
            error_cnt += 1
        #else:
            #print(f"Calculate value: {inBuffer0[i]:.17f}")
    FFT_out.close()
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")

    print("============================")
    print("Exit process")

